In [ ]:
pip install tensorflow keras opencv-python numpy pandas flask streamlit scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load images and masks
def load_data(image_dir, mask_dir, img_size=(256, 256)):
    images = []
    masks = []
    for image_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, image_file)  # Assuming mask filenames are same as images

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        images.append(img)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, img_size)
        mask = mask / 255.0
        masks.append(mask)

    images = np.array(images).reshape(-1, img_size[0], img_size[1], 1)
    masks = np.array(masks).reshape(-1, img_size[0], img_size[1], 1)
    masks = to_categorical(masks, num_classes=2)  # Convert to categorical for segmentation

    return images, masks

image_dir = "/content/dataset"
mask_dir = "/content/dataset"
images, masks = load_data(image_dir, mask_dir)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras import layers, models

def nested_unet(input_shape):
    inputs = layers.Input(input_shape)

    def conv_block(x, filters):
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        return x

    def up_block(x, skip, filters):
        x = layers.Conv2DTranspose(filters, 2, strides=(2, 2), padding='same')(x)
        x = layers.concatenate([x, skip])
        x = conv_block(x, filters)
        return x

    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    bn = conv_block(p4, 1024)

    # Decoder
    u4 = up_block(bn, c4, 512)
    u3 = up_block(u4, c3, 256)
    u2 = up_block(u3, c2, 128)
    u1 = up_block(u2, c1, 64)

    outputs = layers.Conv2D(2, 1, activation='softmax')(u1)

    model = models.Model(inputs, outputs)
    return model

nested_unet_model = nested_unet(input_shape=(256, 256, 1))
nested_unet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras import layers, models

def attention_block(x, g, filters):
    # Keep the shapes of `x` and `g` the same for concatenation.
    theta_x = layers.Conv2D(filters, (1, 1), padding='same')(x)  # Removed downsampling stride
    phi_g = layers.Conv2D(filters, (1, 1), padding='same')(g)
    add_xg = layers.add([theta_x, phi_g])
    relu_xg = layers.Activation('relu')(add_xg)
    psi = layers.Conv2D(1, (1, 1), padding='same')(relu_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    return layers.multiply([x, sigmoid_xg])

def attention_unet(input_shape):
    inputs = layers.Input(input_shape)

    def conv_block(x, filters):
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        return x

    def up_block(x, skip, filters):
        x = layers.Conv2DTranspose(filters, 2, strides=(2, 2), padding='same')(x)
        attn = attention_block(skip, x, filters)
        x = layers.concatenate([x, attn])
        x = conv_block(x, filters)
        return x

    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    bn = conv_block(p4, 1024)

    # Decoder
    u4 = up_block(bn, c4, 512)
    u3 = up_block(u4, c3, 256)
    u2 = up_block(u3, c2, 128)
    u1 = up_block(u2, c1, 64)

    outputs = layers.Conv2D(2, 1, activation='softmax')(u1)

    model = models.Model(inputs, outputs)
    return model

# Compile the model
attention_unet_model = attention_unet(input_shape=(256, 256, 1))
attention_unet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train Nested U-Net
nested_unet_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)

# Train Attention U-Net
attention_unet_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 42s 42s/step - accuracy: 0.1453 - loss: 0.6968 - val_accuracy: 1.0000 - val_loss: 0.6875
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 40s 40s/step - accuracy: 1.0000 - loss: 0.6840 - val_accuracy: 1.0000 - val_loss: 0.6772
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 37s 37s/step - accuracy: 1.0000 - loss: 0.6675 - val_accuracy: 1.0000 - val_loss: 0.6548
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 38s 38s/step - accuracy: 1.0000 - loss: 0.6320 - val_accuracy: 1.0000 - val_loss: 0.5764
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 1.0000 - loss: 0.5039 - val_accuracy: 1.0000 - val_loss: 0.0851
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 1.0000 - loss: 0.0597 - val_accuracy: 1.0000 - val_loss: 1.4390e-07
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 1.0000 - loss: 0.0210 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 1.0000 - loss: 0.2429 - val_accuracy: 1.0000 - val_loss:

In [ ]:
from sklearn.metrics import jaccard_score
import numpy as np

# Reshape the test data
y_test_reshaped = np.argmax(y_test, axis=-1).reshape(-1, 256, 256)

# Predict on test data and reshape
y_pred_nested = np.argmax(nested_unet_model.predict(X_test), axis=-1)
y_pred_attention = np.argmax(attention_unet_model.predict(X_test), axis=-1)

# Flatten both the predictions and the ground truth
y_test_flattened = y_test_reshaped.flatten()
y_pred_nested_flattened = y_pred_nested.flatten()
y_pred_attention_flattened = y_pred_attention.flatten()

# Compute Jaccard similarity score (macro averaged across classes)
dice_nested = jaccard_score(y_test_flattened, y_pred_nested_flattened, average='macro')
dice_attention = jaccard_score(y_test_flattened, y_pred_attention_flattened, average='macro')

print(f"Jaccard Score for Nested U-Net: {dice_nested}")
print(f"Jaccard Score for Attention U-Net: {dice_attention}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Jaccard Score for Nested U-Net: 1.0
Jaccard Score for Attention U-Net: 1.0


In [ ]:
pip install fastapi uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install python-multipart


In [ ]:
!pip install fastapi uvicorn python-multipart


In [ ]:
from fastapi import FastAPI, File, UploadFile
import cv2
import numpy as np

app = FastAPI()

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()

    # Convert file contents to NumPy array
    nparr = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)

    # Resize image to (256, 256) and normalize
    img = cv2.resize(img, (256, 256))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    # Use the appropriate model for prediction
    pred = nested_unet_model.predict(img)
    pred = np.argmax(pred, axis=-1).reshape(256, 256)

    return {"prediction": pred.tolist()}


In [ ]:
!uvicorn fastapi_app:app --reload --host 0.0.0.0 --port 8000


INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [14406] using StatReload
ERROR:    Error loading ASGI app. Could not import module "fastapi_app".
INFO:     Stopping reloader process [14406]


In [ ]:
pip install streamlit requests


In [ ]:
# streamlit_app.py
import streamlit as st
import requests
from PIL import Image
import numpy as np

st.title("Brain MRI Metastasis Segmentation")

uploaded_file = st.file_uploader("Upload a brain MRI image", type="jpg")

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded MRI Image", use_column_width=True)

    # Send image to FastAPI for prediction
    files = {"file": uploaded_file.getvalue()}
    response = requests.post("http://127.0.0.1:8000/predict/", files=files)

    if response.status_code == 200:
        prediction = np.array(response.json()["prediction"])
        st.image(prediction, caption="Predicted Segmentation", use_column_width=True)


2024-10-01 06:16:33.266 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:16:33.446 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-01 06:16:33.452 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:16:33.457 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:16:33.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:16:33.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 06:16:33.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
